In [1455]:
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et
from datetime import datetime
from pandas.core.tools.datetimes import to_datetime
from datetime import timedelta

In [1456]:
def read_xml_data(filename, selected_items):
  tree=et.parse(filename)
  root=tree.getroot()
  #extract selected items
  for child in root:
    if child.tag in selected_items:
      df = pd.DataFrame()
      for elem in child:      
        df1 = pd.DataFrame(elem.attrib, index=[0])
        #df = df.append(df1)
        df = pd.concat([df, df1])
      #First column is the timestamp (dayfirst)      
      #df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], dayfirst=True)
      #write to csv file using the timestamp as index      
      df.to_csv(child.tag+'.csv', index=0)

In [1457]:
def read_ts_file(filename):
  #reads csv file where the first column is a timestamp and the index column
  df = pd.read_csv (filename, parse_dates=[0], dayfirst=True, index_col=0)
  return df

In [1458]:
def align_timeseries(align_df, source_df, source_columns, default_vals, dest_columns=[], include_index=False):
  # Generates a new dataframe with source_df data aligned to align_df
  # source_columns : list of columns from source_df to be included in result
  # dest_columns = column names of source_columns, if ommited column names will be the same as in source_df
  #                if specified, the first column has to be the name of the index in the destination
  # include_index: if source index must be included in the result
  # default_vals = list of default values for source_columns
 
  df_align = align_df.copy()
  df_align = df_align.sort_index()
  df2 = source_df.copy()
  df2 = df2.sort_index()
  df_dup = pd.DataFrame()
  if (dest_columns==[]) : dest_columns = source_columns.copy()
  if (len(dest_columns) == len(source_columns)) : dest_columns = ['source_ts'] + dest_columns # add column for index
  #add blank cells in destination
  df_align[dest_columns] = [np.nan] + default_vals
  for i in range(len(df2)):
      ts2 = df2.index[i]
      ts1_loc = df_align.index.get_loc(ts2, method='nearest')
      ts1 = df_align.index[ts1_loc]
      vals = [ts2]+list(df2.loc[ts2, source_columns])
      ts_old = df_align.loc[ts1, dest_columns[0]]
      if not pd.isna(ts_old): #there are duplicates
        if (abs(ts1.value-ts2.value) > abs(ts1.value-ts_old.value)): 
          #difference is greater, do not replace        
          df_dup = df_dup.append(df_align.loc[ts1])
          df_dup.loc[ts1, dest_columns] = vals
        else: #replace
          df_dup = df_dup.append(df_align.loc[ts1])
          df_align.loc[ts1, dest_columns] = vals
        continue
      df_align.loc[ts1, dest_columns] = vals
  
  if not(include_index) : df_align = df_align.drop( columns = dest_columns[0])

  return df_align, df_dup

In [1459]:
def timedf(df):
  #creates a 5 minute interval timeseries dataframe based in index of df
  # df must have a timestamp index
  #time_df: result dataframe with timestamp index
  timestamp = pd.date_range(start=df.index[0], end=df.index[-1]  + timedelta(minutes=4), freq='5T')
  time_df = pd.DataFrame({'timestamp':timestamp})
  time_df.set_index('timestamp', inplace=True)
  return time_df

In [1460]:
def find_gaps(df, greaterthan=5, units='m'):
  # find gaps relative to index, index must be a datetime field
  # greaterthan is the number of time units to be considered a gap
  # units 'm'=minutes, 'h'=hours
  i = 0
  gaps_df = pd.DataFrame()
  while i < len(df) - 1:
    ts = df.index[i]
    next_ts = df.index[i+1]
    duration = next_ts - ts
    if duration > np.timedelta64(greaterthan, units): 
      begin_gap = ts
      end_gap = next_ts
      gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
    i = i + 1
  gaps_df.sort_values(by=['Duration'], ascending=False, inplace=True)
  return gaps_df

In [1461]:
def read_files():
  #read glucose
  glucose_df = read_ts_file('glucose_level.csv')
  glucose_df.rename(columns={"ts": "timestamp", "value": "glucose"}, inplace=True)
  print(glucose_df)
  #read meal
  meal_df = read_ts_file('meal.csv')
  print(meal_df)
  #read bolus
  meal_df = read_ts_file('bolus.csv')
  print(meal_df)

## Read Training  Data

In [1462]:
#selected_items = ['glucose_level','bolus','meal']
read_xml_data(filename='c://aadm/540-ws-training.xml', selected_items=['glucose_level','bolus','meal'])

In [1463]:
glucose_train = read_ts_file('glucose_level.csv')
glucose_train.rename(columns={"ts": "timestamp", "value": "glucose"}, inplace=True)
glucose_train.describe()

,glucose
count,11947.000000
mean,137.098602
std,55.026251
min,40.000000
25%,93.500000
50%,127.000000
75%,171.000000
max,369.000000


# Explore gaps

In [1464]:
#Finding the length of the complete time series
time_df = timedf(glucose_train)
print('Missing intervals: ', len(time_df) - len(glucose_train) )

Missing intervals:  1162


In [1465]:
# All gaps 
gaps_df = find_gaps(glucose_train)
#print(gaps_df)

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

In [1466]:
#Adding NA in the whole range of cgm-training
glucose_train=glucose_train.resample('5T').mean()
glucose_train.head()

,glucose
ts,
2027-05-19 11:35:00,76.0
2027-05-19 11:40:00,72.0
2027-05-19 11:45:00,68.0
2027-05-19 11:50:00,65.0
2027-05-19 11:55:00,63.0


In [1467]:
#cgmtrainclean= cgmtrainmiss.interpolate(method="linear")
#cgmtrainclean.head()
#missing_minutes =list(df2sp2[df2sp2['glucose'].isna()].index)
#missing_minutes
#cgmtrainclean.describe()

In [1468]:
#import rpy2
#import rpy2.rinterface
#%load_ext rpy2.ipython
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
#splines = importr('splines') 
#x_train=np.arange(len(cgmtrainclean))
#y_train=cgmtrainclean['glucose']
#r_y = robjects.FloatVector(y_train)
#r_x = robjects.FloatVector(x_train)
#r_smooth_spline = robjects.r['smooth.spline'] #extract R function# run smoothing function
#spline1 = r_smooth_spline(x=r_x,y=r_y, spar=.01)
#ySpline=np.array(robjects.r['predict'](spline1,robjects.FloatVector(x_train)).rx2('y'))
#print(ySpline)
#plt.figure(figsize=(12,6))
#plt.scatter(x_train,y_train,c="blue")
#plt.plot(x_train,ySpline,c='red')

## Reading testing data

In [1469]:
#selected_items = ['glucose_level',"bolus','meal']
read_xml_data(filename='c://aadm/540-ws-testing.xml', selected_items=['glucose_level','bolus','meal'])

In [1470]:
glucose_test = read_ts_file('glucose_level.csv')
glucose_test.rename(columns={"ts": "timestamp", "value": "glucose"}, inplace=True)
max=glucose_test.max()
min=glucose_test.min()

In [1471]:
#Finding the length of the complete time series
time_df = timedf(glucose_test)
print('Missing intervals: ', len(time_df) - len(glucose_test) )

Missing intervals:  170


In [1472]:
# All gaps 
gaps_df = find_gaps(glucose_test)
#print(gaps_df)

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
C:\Users\eacun\AppData\Local\Temp\ipykernel_11820\1628313578.py:14: FutureWarning: Th

In [1473]:
#Adding NA in the whole range of cgm-testing
glucose_test=glucose_test.resample('5T').mean()
glucose_test.head()

,glucose
ts,
2027-07-04 00:00:00,254.0
2027-07-04 00:05:00,250.0
2027-07-04 00:10:00,249.0
2027-07-04 00:15:00,247.0
2027-07-04 00:20:00,242.0


In [1474]:
#Normalizing the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
datatrain = np.array(glucose_train.values.astype('float32'))
datatest = np.array(glucose_test.values.astype('float32'))
scaler = MinMaxScaler(feature_range=(0, 1))
datatrain = scaler.fit_transform(datatrain).flatten()
datatest = scaler.fit_transform(datatest).flatten()
#n = len(data)
train_data=pd.DataFrame(datatrain)
test_data=pd.DataFrame(datatest)
print(max,min)

glucose    400
dtype: int64 glucose    49
dtype: int64


In [1475]:
print(train_data.shape)
print(test_data.shape)

(13109, 1)
(3066, 1)


In [1476]:
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars=1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [1477]:
#Past history: One hour, Prediction horizon: 30 minutes 
n_lag=12
n_seq=6

In [1478]:
data1=series_to_supervised(train_data, n_in=n_lag, n_out=n_seq, dropnan=True)
data2=series_to_supervised(test_data, n_in=n_lag, n_out=n_seq, dropnan=True)
train=data1.values
test=data2.values
#print(data1.shape)
print("test shape:",test.shape)

test shape: (2743, 18)


In [1479]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers import TimeDistributed
from keras.layers.convolutional import MaxPooling1D
import datetime   
import math

In [1480]:
n_features=1
timesteps=n_lag
X, y = train[:, 0:n_lag], train[:, n_lag:]
print(type(X))
print(X.shape)
print(y.shape)
rows_x = len(y)
#print("rows-x=",rows_x)
#print(trainX.shape)
X = np.reshape(X, (rows_x, timesteps, 1)) 
print(X.shape)
#print('train-train',X.shape)
#print('train-test',y.shape)
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timesteps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_seq))
model.summary()
model.compile(optimizer='adam', loss='mse')

<class 'numpy.ndarray'>
(11419, 12)
(11419, 6)
(11419, 12, 1)
Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_44 (Conv1D)          (None, 11, 64)            192       
                                                                 
 max_pooling1d_44 (MaxPoolin  (None, 5, 64)            0         
 g1D)                                                            
                                                                 
 flatten_44 (Flatten)        (None, 320)               0         
                                                                 
 dense_88 (Dense)            (None, 50)                16050     
                                                                 
 dense_89 (Dense)            (None, 6)                 306       
                                                                 
Total params: 16,548
Trainable params: 16,548
Non-trainab

In [1481]:
import time
start_time = time.time()
model.fit(X, y, epochs=100,batch_size=32,validation_split=.2,verbose=2)

Epoch 1/100
286/286 - 1s - loss: 0.0092 - val_loss: 0.0043 - 812ms/epoch - 3ms/step
Epoch 2/100
286/286 - 0s - loss: 0.0044 - val_loss: 0.0036 - 422ms/epoch - 1ms/step
Epoch 3/100
286/286 - 0s - loss: 0.0038 - val_loss: 0.0033 - 437ms/epoch - 2ms/step
Epoch 4/100
286/286 - 0s - loss: 0.0035 - val_loss: 0.0031 - 447ms/epoch - 2ms/step
Epoch 5/100
286/286 - 0s - loss: 0.0033 - val_loss: 0.0031 - 437ms/epoch - 2ms/step
Epoch 6/100
286/286 - 0s - loss: 0.0032 - val_loss: 0.0029 - 437ms/epoch - 2ms/step
Epoch 7/100
286/286 - 0s - loss: 0.0031 - val_loss: 0.0028 - 450ms/epoch - 2ms/step
Epoch 8/100
286/286 - 0s - loss: 0.0031 - val_loss: 0.0029 - 370ms/epoch - 1ms/step
Epoch 9/100
286/286 - 0s - loss: 0.0030 - val_loss: 0.0030 - 406ms/epoch - 1ms/step
Epoch 10/100
286/286 - 0s - loss: 0.0030 - val_loss: 0.0028 - 375ms/epoch - 1ms/step
Epoch 11/100
286/286 - 0s - loss: 0.0029 - val_loss: 0.0028 - 499ms/epoch - 2ms/step
Epoch 12/100
286/286 - 0s - loss: 0.0029 - val_loss: 0.0027 - 375ms/epoch 

Epoch 98/100
286/286 - 0s - loss: 0.0026 - val_loss: 0.0026 - 367ms/epoch - 1ms/step
Epoch 99/100
286/286 - 0s - loss: 0.0026 - val_loss: 0.0027 - 382ms/epoch - 1ms/step
Epoch 100/100
286/286 - 0s - loss: 0.0027 - val_loss: 0.0027 - 383ms/epoch - 1ms/step


In [1482]:
#Normalized Testing predictions 
xt=test[:,0:12]
xt=np.reshape(xt,(xt.shape[0],12,1))
a=model.predict(xt,batch_size=n_seq)
print(a)

458/458 [==============================] - 0s 743us/step
[[0.4529918  0.44549775 0.4382415  0.43255693 0.42879722 0.4231955 ]
 [0.4441399  0.4379146  0.43174818 0.42674607 0.4233919  0.41881537]
 [0.43285632 0.42579472 0.4200045  0.41509247 0.41126895 0.4075381 ]
 ...
 [0.36655214 0.38443154 0.3957423  0.40357298 0.40590224 0.40484393]
 [0.41971052 0.43783402 0.44892794 0.45500857 0.45429087 0.4499446 ]
 [0.4007807  0.4063909  0.40450594 0.4008149  0.3956318  0.38527387]]


In [1483]:
max=np.array(max)
min=np.array(min)

In [1484]:
#Denormalizing prediction
forec1=a*(max-min)+min
print(forec1)

[[208.00012651 205.36971068 202.82277057 200.82748145 199.50782254
  197.5416244 ]
 [204.89310414 202.70802808 200.54361174 198.78787071 197.61055985
  196.00419641]
 [200.93256879 198.45394695 196.42157495 194.69745636 193.35540128
  192.04586825]
 ...
 [177.65980265 183.93547088 187.90554631 190.65411478 191.47168508
  191.10021818]
 [196.31839144 202.67974257 206.57370657 208.7080068  208.45609426
  206.93054953]
 [189.67402837 191.64320779 190.98158434 189.68602669 187.86675835
  184.23112971]]


In [1485]:
#Denomalizing the actual values
test=pd.DataFrame(test)
testy=test.loc[:,12:18]
actual=np.array(testy)*(max-min)+min
print(actual)

[[208.99999395 205.00000116 208.00000098 210.00000784 205.99999413
  201.00000837]
 [205.00000116 208.00000098 210.00000784 205.99999413 201.00000837
  198.00000855]
 [208.00000098 210.00000784 205.99999413 201.00000837 198.00000855
  195.00000873]
 ...
 [181.00000259 205.00000116 225.0000174  239.99999559 249.00001597
  253.99998081]
 [205.00000116 225.0000174  239.99999559 249.00001597 253.99998081
  247.99998116]
 [225.0000174  239.99999559 249.00001597 253.99998081 247.99998116
  241.99998152]]


In [1486]:
#Computing the RMSE
forec1=np.array(forec1)
actual=np.array(actual)
diff=actual-forec1
#print(diff.shape)
#np.sqrt(np.mean((diff)**2,axis=0))
print("RMSE:",np.sqrt(np.mean((diff)**2,axis=0)).mean())

RMSE: 16.916969989126997


In [1487]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 40.78574585914612 seconds ---
